# Chat

`Chat` is an object for conversational LLM interactions that tracks history and token usage across single or multiple models.

In [ ]:
from irouter import Chat

# To load OPENROUTER_API_KEY from .env file create a .env file at the root of the project with OPENROUTER_API_KEY=your_api_key
# Alternatively pass api_key=your_api_key to the Chat class
from dotenv import load_dotenv
load_dotenv();

True

In this notebook we will use free tiers for Moonshot AI's Kimi K2 and Google's Gemma 3N. 

An overview of all available models can be discovered with `get_all_models`:
```python
from irouter.base import get_all_models
model_slugs = get_all_models()
model_slugs
```

You can also browse available models at [openrouter.ai/models](https://openrouter.ai/models).

In [2]:
model_names = ["moonshotai/kimi-k2:free", "z-ai/glm-4.5-air:free"]

# Single Model

The simplest way to use `Chat` is with a single LLM by providing a model slug. Unlike `Call`, `Chat` maintains conversation history and tracks token usage.

In this example we initialize a `Chat` object with the free tier of Moonshot AI's Kimi-K2 LLM.

To set the API key you can either set an environment variable for `OPENROUTER_API_KEY` to your project or pass `api_key` when initializing `Chat`.

In [3]:
c = Chat(model_names[0], system="You are the best assistant in the world.")
# or
# c = Chat(model_names[0], api_key="your_api_key")

At the start the `history` will only contain the system message.

In [4]:
c.history

[{'role': 'system', 'content': 'You are the best assistant in the world.'}]

`Chat` will also tracks the token usage.

In [5]:
c.usage

{'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}

In [6]:
c("Who created you?")

'I was created by Moonshot AI.'

After each call the `history` and `usage` is updated.

In [7]:
c.history

[{'role': 'system', 'content': 'You are the best assistant in the world.'},
 {'role': 'user', 'content': 'Who created you?'},
 {'role': 'assistant', 'content': 'I was created by Moonshot AI.'}]

In [8]:
c.usage

{'prompt_tokens': 24, 'completion_tokens': 9, 'total_tokens': 33}

# Multiple LLMs

In [9]:
c = Chat(model_names, system="You are the best assistant in the world.")

If multiple LLMs are used, we define the `history` and `usage` as a dictionary mapping from the LLM slug.

In [10]:
c.history

{'moonshotai/kimi-k2:free': [{'role': 'system',
   'content': 'You are the best assistant in the world.'}],
 'z-ai/glm-4.5-air:free': [{'role': 'system',
   'content': 'You are the best assistant in the world.'}]}

In [11]:
c.usage

{'moonshotai/kimi-k2:free': {'prompt_tokens': 0,
  'completion_tokens': 0,
  'total_tokens': 0},
 'z-ai/glm-4.5-air:free': {'prompt_tokens': 0,
  'completion_tokens': 0,
  'total_tokens': 0}}

In [12]:
c("Who created you?")

{'moonshotai/kimi-k2:free': 'I was created by Moonshot AI.',
 'z-ai/glm-4.5-air:free': "\nI'm GLM, a large language model created and trained by Zhipu AI. I'm designed to be helpful, informative, and engaging in conversations while respecting privacy and safety guidelines.\n\nIs there something specific about my capabilities or origins you'd like to know more about?"}

irouter's `Chat` will keep separate track of each model's history and usage. In this way you can have multi-turn conversations with multiple models at the same time and can analyze where each model ends up.

In [13]:
c.history

{'moonshotai/kimi-k2:free': [{'role': 'system',
   'content': 'You are the best assistant in the world.'},
  {'role': 'user', 'content': 'Who created you?'},
  {'role': 'assistant', 'content': 'I was created by Moonshot AI.'}],
 'z-ai/glm-4.5-air:free': [{'role': 'system',
   'content': 'You are the best assistant in the world.'},
  {'role': 'user', 'content': 'Who created you?'},
  {'role': 'assistant',
   'content': "\nI'm GLM, a large language model created and trained by Zhipu AI. I'm designed to be helpful, informative, and engaging in conversations while respecting privacy and safety guidelines.\n\nIs there something specific about my capabilities or origins you'd like to know more about?"}]}

In [14]:
c.history["moonshotai/kimi-k2:free"]

[{'role': 'system', 'content': 'You are the best assistant in the world.'},
 {'role': 'user', 'content': 'Who created you?'},
 {'role': 'assistant', 'content': 'I was created by Moonshot AI.'}]

For this particular query, Z.ai's GLM 4.5 Air used more tokens compared to Moonshot AI's Kimi K2. 

In [15]:
c.usage

{'moonshotai/kimi-k2:free': {'prompt_tokens': 24,
  'completion_tokens': 9,
  'total_tokens': 33},
 'z-ai/glm-4.5-air:free': {'prompt_tokens': 20,
  'completion_tokens': 155,
  'total_tokens': 175}}

In [16]:
c.usage["moonshotai/kimi-k2:free"]

{'prompt_tokens': 24, 'completion_tokens': 9, 'total_tokens': 33}

# Resetting history and usage

History can be reset by calling `reset_history`. The `Chat` object history will revert to the system prompt.

In [17]:
c.history

{'moonshotai/kimi-k2:free': [{'role': 'system',
   'content': 'You are the best assistant in the world.'},
  {'role': 'user', 'content': 'Who created you?'},
  {'role': 'assistant', 'content': 'I was created by Moonshot AI.'}],
 'z-ai/glm-4.5-air:free': [{'role': 'system',
   'content': 'You are the best assistant in the world.'},
  {'role': 'user', 'content': 'Who created you?'},
  {'role': 'assistant',
   'content': "\nI'm GLM, a large language model created and trained by Zhipu AI. I'm designed to be helpful, informative, and engaging in conversations while respecting privacy and safety guidelines.\n\nIs there something specific about my capabilities or origins you'd like to know more about?"}]}

In [18]:
c.reset_history()

In [19]:
c.history

{'moonshotai/kimi-k2:free': [{'role': 'system',
   'content': 'You are the best assistant in the world.'}],
 'z-ai/glm-4.5-air:free': [{'role': 'system',
   'content': 'You are the best assistant in the world.'}]}

Usage can be reset with `reset_usage`.

In [20]:
c.usage

{'moonshotai/kimi-k2:free': {'prompt_tokens': 24,
  'completion_tokens': 9,
  'total_tokens': 33},
 'z-ai/glm-4.5-air:free': {'prompt_tokens': 20,
  'completion_tokens': 155,
  'total_tokens': 175}}

In [21]:
c.reset_usage()

In [22]:
c.usage

{'moonshotai/kimi-k2:free': {'prompt_tokens': 0,
  'completion_tokens': 0,
  'total_tokens': 0},
 'z-ai/glm-4.5-air:free': {'prompt_tokens': 0,
  'completion_tokens': 0,
  'total_tokens': 0}}

I hope this gives you a good overview of the basic usage of `Chat`. Check `img.ipynb`, `pdf.ipynb` and `audio.ipynb` for examples on using `Chat` with other modalities.